In [2]:
# Python Version 3.8.10
#Importing the libraries

import numpy as np
import PIL 
from PIL import Image
import random
import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader , Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.utils as vutils
import PIL.ImageOps as ImageOps
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
#Creating helper functions to visualize images and losses

def image_show(img , text = None ,save = False):
    ''' This Function plots the image.'''
    npimg = img.numpy()
    plt.axis("off")
    
    if text is not None: #if there is text to insert
        plt.text(75,8 , text , style= "italic" , fontweight = "bold" , bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg , (1,2,0)))
    plt.show()

def plot_loss(iteration , loss):
    plt.plot(iteration , loss)
    plt.grid(True)
    plt.xlabel("ITERATIONS")
    plt.ylabel("LOSS")
    plt.show()

In [8]:
class CustomDataset(Dataset):
    
    def __init__(self,image_folders,transform=None,should_invert=True):
        self.image_folders = image_folders    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.image_folders.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.image_folders.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.image_folders.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.image_folders.imgs)

In [ ]:
#Defining the paths for training and testing data
training_dir = "/content/drive/My Drive/Siamse/Siamese/data/faces/training/"
testing_dir = "/content/drive/My Drive/Siamse/Siamese/data/faces/testing/"

folder_dataset = dset.ImageFolder(root = training_dir)